# <font color='red'>Basic Analytics Applied to Training Set of Heroes of the Storm Game Data</font>

### 1. Determine the best Heroes(in terms of win rate) in each category:
* Support
* Assassin
* Warrior
* Specialist

### 2. Determine the best Team Composition in term of win rate

### 3. Does the map affect the team composition?

### 4. Which Assassin is better; ranged or melee?

### 5. What are the most played Heroes in each category?  Are they the most likely heroes to win?




# if dataset too large:
    ## reduce to only Hero League: ~1.7million lines
    ## reduce to Quick Match: ~3.5 million lines 

In [1]:
#import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import csv
import sys


#load both csv files into tow respective dataframes
DF_Replay = pd.read_csv("TSReplay.csv")

DF_Characters = pd.read_csv("TSCharacters.csv")

DF_Map_Groups = pd.read_csv("HeroAndMap.csv")

In [2]:
DF_Replay.head(2)

,ReplayID,GameMode(3=Quick Match 4=Hero League 5=Team League),Map,Replay Length,Timestamp (UTC)
0,23915730,4,Haunted Mines,00:12:00,5/14/2015 12:55:26 AM
1,23927005,3,Cursed Hollow,00:21:00,5/14/2015 12:55:26 AM


In [3]:
#Split the string for ReplayLength into several sub-cells based on the ':'
# XX : YY : ZZ
# X *60 *60 (for float(sec))
# Y *1 *60  (for float(sec))
# Z *1      (for float(sec))

#from format   |  hh:mm:mm  |
#to format     |  hh  |  :  | mm:ss  |
ChangeTime = DF_Replay['Replay Length'].str.partition(':')
ChangeTime.columns = ['Hour', 'Col2', 'MinSec']
#Now we have one column for hours, and one for both minutes and seconds to partition again

#from format  |  mm:ss  |
#to format    |  mm  |  :  |  ss  |
ChangeTime2 = ChangeTime['MinSec'].str.partition(':')
ChangeTime2.columns = ['Min', 'Space', 'Second']
#now we have one column for minutes and one for seconds

#Get both column in float type
ChangeTime2['Min'] = ChangeTime2['Min'].astype(float)
ChangeTime2['Second'] = ChangeTime2['Second'].astype(float)

#Apply math to convert min to sec, then add minutes and seconds together
ChangeTime2['Min']= ChangeTime2['Min'] * 60
ChangeTime2['Second'] = ChangeTime2['Min'] + ChangeTime2['Second']
ChangeTime2 = ChangeTime2[[2]]

ChangeTime = ChangeTime.drop('Col2', axis=1).drop('MinSec', axis=1)
ChangeTime['Hour'] = ChangeTime['Hour'].astype(float)
#change the values in the column with *60 to get in minutes, and *60 again to get result in seconds
ChangeTime['Hour'] = ChangeTime['Hour'] * 60 * 60     

#ChangeTime.head(10)
ChangeTime2.head(10)

#Add the 2 int values in each cell to get final number of days
#FinalTime = ChangeTime + ChangeTime2
#FinalTime.head(10) 

#DataFrame['last_tweet'] = Attempt 

,Second
0,720
1,1260
2,1578
3,1146
4,1378
5,1059
6,983
7,995
8,1311
9,1254


In [4]:
DF_Characters.head(2)

,ReplayID,Is Auto Select,Hero,Hero Level,Is Winner,MMR Before
0,23915730,False,Rehgar,14,False,3170
1,23915730,False,Falstad,10,False,3330


In [5]:
DF_Map_Groups.head(2)

,ID,Name,Group,SubGroup
0,0,Unknown,NaN,NaN
1,1,Abathur,Specialist,Utility


In [6]:
#Remove all after row 49, to remove map IDs and keep all data for Heroes
DF_Map_Groups = DF_Map_Groups[:48]

#DF3 = DF.drop('ID', axis=1).drop('SubGroup', axis=1)
DF_Map_Groups = DF_Map_Groups.drop('SubGroup', axis=1).drop('ID', axis=1)

#Returning last 3 elements to check
DF_Map_Groups.iloc[45:]

,Name,Group
45,Gall,Assassin
46,Lunara,Assassin
47,Greymane,Assassin


In [7]:
#Need to merge on 'ReplayID' and duplicate entries
from pandas import merge
DF1 = pd.merge(DF_Replay, DF_Characters, left_on=["ReplayID"], right_on=['ReplayID'], how='left')
DF1.head(4)

#deleting previous dataframes created for memory reasons
del DF_Replay
del DF_Characters

In [8]:
#Need to merge on 'Name' 
DF = pd.merge(DF1, DF_Map_Groups, left_on=["Hero"], right_on=["Name"], how='left')

#delete the former dataframe DF_Map_Groups for memory's optimization
del DF_Map_Groups
del DF1

In [9]:
#Saving the merged Dataframe to a csv file
#DF.to_csv("MergedData.csv")

In [10]:
#DataFrame = pd.read_csv("MergedData.csv")

In [11]:
list(DF.columns.values)

['ReplayID',
 'GameMode(3=Quick Match 4=Hero League 5=Team League)',
 'Map',
 'Replay Length',
 'Timestamp (UTC)',
 'Is Auto Select',
 'Hero',
 'Hero Level',
 'Is Winner',
 'MMR Before',
 'Name',
 'Group']

In [12]:
DF.dtypes 
#ReplayID          int64     >>>Change to Category
#GameMode          int64     >>>Replace with Category (GAME TYPE)
#Map              object
#ReplayLength     object
#Timestamp        object
#AutoSelect         bool
#Hero             object
#HeroLevel         int64
#IsWinner           bool
#MMRBefore       float64
#Name             object
#Group            object

ReplayID                                                 int64
GameMode(3=Quick Match 4=Hero League 5=Team League)      int64
Map                                                     object
Replay Length                                           object
Timestamp (UTC)                                         object
Is Auto Select                                            bool
Hero                                                    object
Hero Level                                               int64
Is Winner                                                 bool
MMR Before                                             float64
Name                                                    object
Group                                                   object
dtype: object

In [13]:
#Overwrite the DataFrame's Column names to get it into correct format for future use
DF.columns = ['ReplayID', 'GameMode', 'Map', 'ReplayLength', 'Timestamp', 'AutoSelect', 'Hero', 'HeroLevel', 'IsWinner', 'MMRBefore', 'Name', 'Group']
#similar to:
#DF = DF.rename(columns = ('Replay Length':'ReplayLength'))
#DF = DF.rename(columns = {'Timestamp (UTC)':'Timestamp'})
#DF = DF.rename(columns = {'Is Auto Select':'AutoSelect'})
#...

DF.head(1)

,ReplayID,GameMode,Map,ReplayLength,Timestamp,AutoSelect,Hero,HeroLevel,IsWinner,MMRBefore,Name,Group
0,23915730,4,Haunted Mines,00:12:00,5/14/2015 12:55:26 AM,False,Rehgar,14,False,3170,Rehgar,Support


In [15]:
#numeric = DF.select_dtypes(['int64','float64']).columns
#DF[numeric].head(5)

In [16]:
DF[['GameMode', 'HeroLevel', "MMRBefore"]].corr()

MemoryError: 

In [ ]:
%matplotlib inline
DF[numeric].hist(figsize=(20, 20), color='red')
#plt.savefig("Histogram-Numeric-Features_HOTS.png")

In [ ]:
#Normalize Replay Lenght to have all in int(SECONDS) or float(MINUTES)
#look at the homework_2 for code on parsing based on " " and format covnersion to int or float

# parse and multiply the number and location


In [ ]:
%matplotlib inline
dfHero = DF[["Hero", "IsWinner"]]
dfHero = dfHero.groupby(["Hero","IsWinner"])["Hero"].count().unstack('IsWinner')
dfHero.plot(kind="bar", stacked=True, figsize=(15,8), title="Bar Plot for Win/Loss per hero")

In [ ]:
# Exploratory Data Science
    # Why most win = Jaina and Valla  
    #    &
    # Most lost = Jaina and Valla
        #--> Most games = QuickMatch, both team can have Valla & Jaina at sme time!
    
# Define TRAIN and TEST data sets (80-20? 70-30?)
    # define model
    # look into OSL, Kernel Density Estimation
    # Linear Regression
    # logistic Regression
    
# Establish the likeliness of winning based on team rate


# How is auto-select affecting win rate?
# More/Less than Hero level??